## GAHN Alioune Badara Ba
## DIALLO Aicha Aminata

In [52]:
import pandas as pd
import numpy as np

Chargeons les données et Affichons les 5 premières Pages du wikipédia

In [2]:
wiki = pd.read_csv('/home/sid2018-1/Bureau/simplewiki.csv')

In [3]:
wiki1000= wiki[['title','text']]

In [4]:
wiki1000.head()

,title,text
0,Demo (software),A '''demo''' is a small portion or sample of a...
1,Category:Islands of North America,{{Commonscat|Islands of North America}}\n\n[[C...
2,Wikipedia:Mirrors and forks,'''Mirrors and forks of Wikipedia''' are publi...
3,Buck the World,{{italictitle}}\n\n'''''Buck the World''''' is...
4,Naava Nabagesera,{{multiple issues|\n{{BLP unsourced|date=Augus...


Nettoyons maintenant les données à l'aide de la fonction suivante:

In [5]:
def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if (x.isalpha() ) else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

In [6]:
wiki1000['text'] = [sentense2cleanTokens(text) for text in wiki1000['text'].values]
wiki1000['title'] = [sentense2cleanTokens(text) for text in wiki1000['title'].values]

/home/sid2018-1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/sid2018-1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Concaténons maintenant les titres et les textes pour chaque page

In [7]:
wiki1000['doc'] = [wiki1000['title'].values[i] +' '+ wiki1000['text'].values[i] for i in range(len(wiki1000))]

In [8]:
wiki1000.head()

,title,text,doc
0,demo software,a demo is a small portion or sample of a produ...,demo software a demo is a small portion or sam...
1,category islands of north america,commonscat islands of north america category i...,category islands of north america commonscat i...
2,wikipedia mirrors and forks,mirrors and forks of wikipedia are publication...,wikipedia mirrors and forks mirrors and forks ...
3,buck the world,italictitle buck the world is the second album...,buck the world italictitle buck the world is t...
4,naava nabagesera,multiple issues blp unsourced date august comp...,naava nabagesera multiple issues blp unsourced...


In [13]:
for i,k in enumerate (wiki1000['doc'].values):
    if 'leptodactylidae' in k:
        print(i)

3668
4418
92207
95955
101463
110049
158934
171563
215391


Nous n'avons pas assez de mémoire pour charger les frequences de toutes les pages de Wikipedia. Nous allons donc choisir 
les documents ci dessus ou on retrouve le mots 'leptodactylidae' et les 1000 premieres pages pour calculer les TF des mots

In [17]:
wikireduit = wiki1000.iloc[list(range(1001))+[3668,4418,92207,95955,101463,110049,158934,171563,215391]]

#### Question 2)  Les frequences des mots

Ici nous faisons un Countvectorizer pour calculer la frequence des mots

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
tf = CountVectorizer(stop_words = 'english') 
tf_features = tf.fit_transform(wikireduit['doc'].values)

In [26]:
tfdf0 = pd.DataFrame(tf_features.todense(), columns= tf.get_feature_names())

#### Question 3) Calcul de Similarité en utilisant La formule d'ESA

In [33]:
#fonction permettant de calculer le cosine similarity
from scipy import spatial
def simi(v1, v2):
    return 1 - spatial.distance.cosine(v1,v2)

Calculons maintenant la similarité entre les mots en utilisant la fonction précédente

In [34]:
simi(tfdf0['leptodactylidae'].values,tfdf0['dog'].values),simi(tfdf0['leptodactylidae'].values,tfdf0['frog'].values),simi(tfdf0['leptodactylidae'].values,tfdf0['fish'].values),simi(tfdf0['leptodactylidae'].values,tfdf0['bird'].values)

(0.0, 0.30050167725664245, 0.042781360371560884, 0.0)

(dog : 0.0, frog: 0.30050167725664245, fish: 0.042781360371560884, bird: 0.0)

D'aprés la méthode d'ESA nous remarquons que le mots 'leptodactylidae' est beaucoup plus proche de 'frog' que les autres 
mots. Mais  avec une proba trés petite pour 'fish' et nulle pour 'bird' et 'dog'

##### Question 4) Calcul des Similarités en Utilisant les Words Embeddings

In [68]:
# Chargement des poids pré-entrainés
embeddings_index = dict()
f = open('/home/sid2018-1/Téléchargements/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [69]:
w2v =embeddings_index

In [81]:
# Vecteur correspondant à la représentation du mots 
w2v['leptodactylidae']

array([ 0.61412,  1.683  , -1.1209 ,  2.0704 ,  1.7839 , -0.53265,
        0.99716,  0.97641,  1.8206 , -0.84315, -0.4458 , -1.1338 ,
        2.1635 , -0.2002 ,  1.4517 ,  0.9435 , -1.2476 ,  1.314  ,
        0.1291 ,  1.4752 , -1.1792 , -0.84888, -0.35244,  0.79328,
        0.52529, -0.08322, -1.1829 ,  0.61174,  0.09527,  0.2358 ,
       -0.38497, -0.23101,  0.34109,  0.50077, -0.92378,  0.77414,
       -0.28542, -0.4666 ,  0.76878, -1.9943 , -1.1997 , -0.63895,
       -1.7485 ,  0.97641,  0.66343,  0.1826 ,  0.53183, -2.8687 ,
       -0.38984, -0.77487], dtype=float32)

Calculons maintenant les similarités entre les mots en utilisant toujours la similarité Cosinus comme mètrique

In [123]:
simi(w2v['leptodactylidae'],w2v['dog']),simi(w2v['leptodactylidae'],w2v['frog']),simi(w2v['leptodactylidae'],w2v['fish']),simi(w2v['leptodactylidae'],w2v['bird'])

(0.29905593395233154,
 0.5979495048522949,
 0.19104993343353271,
 0.3037750720977783)

(dog : 0.29905593395233154,
 frog: 0.5979495048522949,
 fish: 0.19104993343353271,
 bird: 0.3037750720977783)

D'aprés la méthode des Words Embeddings nous remarquons que le mots 'leptodactylidae' est beaucoup plus proche de 'frog' avec une 
proba associé à 0.6. Ensuite nous avons une proximité du mots avec 'bird' et 'dog' estiméé à environ 0.3. Avec cette méthode, le mots est moins proche de 'fish' 

Nous remarquons que les resultats observés avec les techniques de Words Embeddings sont plus réalistes. Ceci peut s'expliquer
par le fait que ces techniques encapsulent dans leur représentation vectorielle l'aspect semantique des mots tandis que les
techniques d'ESA ne sont basées que sur l'aspect présentiel (frequence du mots dans un corpus etc). 

### 5 ) Pour obtenir une meilleure évaluation de ces méthodes, appliquons les sur le dataset suivant et evaluons les résultats

Lisons d'abord le fichier:

In [41]:
import pandas as pd

In [55]:
trn = pd.read_csv("/home/sid2018-1/Bureau/semeval2014task3PhraseWord/training/phrase2word.train.input.tsv", sep = "\t", header= None)

In [56]:
trn

,0,1,2
0,comply with international obligations,diplomacy,Newswire-1
1,very tedious,tenacity,Newswire-2
2,not my brother,unrelated,Newswire-3
3,able to function,necessary,Newswire-4
4,$50 million in equal installments,defending,Newswire-5
5,obligations to the bank,loan,Newswire-6
6,depending on participation in incentive schemes,commission,Newswire-7
7,vicious cycle of violence,knife,Newswire-8
8,hijacking a Cuban ferry a day later,Hemingway,Newswire-9
9,not the bad guy,nice,Newswire-10


Maintenant Appliquons les différentes méthodes. Nous ne faisons aucun pré-traitement sur les données

### ESA


Pour évaluer les similarités entre les documents nous calculons leur représentation vectorielle en utilisant le CountVectorizer comme 
précedemment

In [76]:
# Nous concatenons les documents des deux colonnes du dataframe à comparer
docs= list(trn[0].values) + list(trn[1].values)

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
tf = CountVectorizer(stop_words = 'english') 
tf_features = tf.fit_transform(docs)

In [78]:
tfdf = pd.DataFrame(tf_features.todense(), columns= tf.get_feature_names())

Calcuons maintenant les similarités obtenus entre documents et enregistrons le fichier output

In [79]:
simiesa = []
for i in range(500):
    simiesa.append(simi(tfdf.iloc[i].values,tfdf.iloc[i+500].values))

/home/sid2018-1/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [80]:
dfesa = pd.DataFrame(simiesa)

In [81]:
np.savetxt("/home/sid2018-1/Bureau/semeval2014task3PhraseWord/training/output_esa.txt", dfesa[0].values)

### Wordnet

Pour calculer la similarité entre deux documents A et B en utilisant Wordnet, la méthode que nous implémentons est la suivante:
    - Pour chaque mots m du document A nous choisissons son premier sens
    - Nous calculons la path_similarity entre le mots m du document A et tous les mots du document B puis nous gardons le score **max**
    - Nous obtenons le score de similarité entre les 2 document en faisant la somme des scores **max** retenus pour chaque mots m du document A

Le code suivant nous permets d'obtenir les similarités entre pairs de documents en utilisant Wordnet

In [64]:
from nltk.corpus import wordnet

In [82]:
simiwn1 = []
for j in trn.index:
    s= 0
    for k in trn.iloc[j][0].split():
        l = []
        for b in trn.iloc[j][1].split():
            if ((wordnet.synsets(k) == []) or (wordnet.synsets(b)== [])):
                l.append(0)
            else:
                o = wordnet.synsets(k)[0].path_similarity(wordnet.synsets(b)[0])
                if (o == None):
                    l.append(0)
                else:
                    l.append(o)
        s = s + max(l)
    simiwn1.append(s)
        

In [84]:
len(trn.index)

500

In [85]:
np.savetxt("/home/sid2018-1/Bureau/semeval2014task3PhraseWord/training/output_wn1.txt", simiwn1)

### Word Embeddings

Pour calculer la similarité des documents en utilisant les Word Embeddings, nous representatons chaque document 
comme la moyenne des vecteurs des mots qu'ils contiennent. 
Ensuite nous utilisons la mètrique cosine similarity pour calculer la similarité entre les documents

Le code suivant nous permet de réaliser cela : 

In [70]:
simiwe = []
keys = w2v.keys()
for j in trn.index:
    s= np.zeros(50)
    for k in trn.iloc[j][0].split():
        if k in keys:
            s = s + w2v[k]
    s = s/ len(trn.iloc[j][0].split())
    if trn.iloc[j][1] in keys:
        o = simi(s,w2v[trn.iloc[j][1]])
    else:
        o = 0
    simiwe.append(o)

In [71]:
simiwe

[0.5290095439688809,
 0.2564507785569783,
 0.3211098175904279,
 0.8661989214804419,
 0.3163422760570104,
 0.6636136116275388,
 0.5877288790198376,
 0.3840052848723854,
 0,
 0.6995571721460859,
 0.2272627224823498,
 0,
 0.20593793050202436,
 0.32963340559377596,
 0.7874864939591018,
 0.7323654812505035,
 0.6806105595016555,
 0.6923088442139999,
 0.590636752833018,
 0.6946752155429905,
 0,
 0.6268824840034013,
 0,
 0.6901695254770752,
 -0.25486230338401716,
 0.3820008029079669,
 0.8242282889933643,
 0.4864904223213311,
 0.5118773290538747,
 0.1236285019613651,
 0.6189317045751291,
 0.5140457728030826,
 0.7159390919855201,
 0.6524854870774578,
 0.6432258859375479,
 0.44289199450293393,
 0.3372115994763607,
 0.18554185852678384,
 0.5294945534177256,
 0,
 0.2576943863607084,
 0.7403643636421471,
 0.09338912897628948,
 0.7170577089611464,
 0.4379594667790181,
 0.3183604871164789,
 0.6871286310692771,
 0.49886927322146235,
 0.5644270458106635,
 0.7402950413092664,
 0.7908841936584042,
 0.2239

In [72]:
np.savetxt("/home/sid2018-1/Bureau/semeval2014task3PhraseWord/training/output_we.txt", simiwe)

**Les résultats obtenus sont les suivants**

baseline : ``Scores for SemEval-2014 Task 3
Pearson's correlation	0,164760	Spearman's rho	0,161558``

ESA : ``Scores for SemEval-2014 Task 3
Pearson's correlation	-0,020459	Spearman's rho	-0,033360``

Wordnet : ``Scores for SemEval-2014 Task 3
Pearson's correlation	-0,006583	Spearman's rho	0,019882``

Word Embeddings : ``Scores for SemEval-2014 Task 3
Pearson's correlation	-0,046106	Spearman's rho	-0,050783``

Les mètriques utilisées ici sont les corrélation de Pearson et Spearman. 
Les coefficients de corrélation permettent de donner une mesure synthétique de l'intensité de la relation entre deux caractères et de son sens lorsque cette relation est monotone. Le coefficient de corrélation de Pearson permet d'analyser les relations linéaires et le coefficient de corrélation de Spearman les relations non-linéaires monotones

Ici en faisant une interprétation avec les valeurs obtenues pour ces corrélations , nous voyons que WordNet marche le mieux, suivi de l'ESA et enfin les techniques de Word Embeddings. 
C'est un peu logique car on a prit le maximum des similarité entre les mots des deux documents pour wordNet tandis qu'on prend la moyenne des words embeddings. 
Une facon d'ameliorer la représentation des document avec les word embedding serait de faire un maxpooling sur chaque composante de la matrice obtenue en concatenant les embeddings de chaque mots pour un document. 

**Conclusion**

Ce TP a été trés interessant car il nous a permis d'illustrer les différentes méthodes et d'obtenir une évaluation comparative de ces techniques. 
Cela nous a permis de voir ce qui marche le mieux lorsqu'on évalue la similarité entre mots et documents 